<a href="https://colab.research.google.com/github/soerenetler/spacy_target_answer/blob/master/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import os
os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [32]:
!(cd spacy_target_answer; git pull)
!(cd squad-question-generation/; git pull)

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/soerenetler/spacy_target_answer
   68f6e42..725c19d  master     -> origin/master
Updating 68f6e42..725c19d
Fast-forward
 setup.ipynb | 208 ++++++++++++++++++++++++++++++++++++++++--------------------
 1 file changed, 140 insertions(+), 68 deletions(-)
Already up to date.


In [33]:
!cp crf_target_answer/Custom.py Custom.py

In [34]:
!nvidia-smi

Sat Jun  5 19:56:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    33W / 250W |   3349MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [35]:
#!pip install --quiet -U pip setuptools wheel

In [36]:
!pip install --quiet -U spacy[cuda110,transformers,lookups]

In [37]:
!python -m spacy download en_core_web_sm --quiet
!python -m spacy download en_core_web_trf --quiet
!python -m spacy download en_core_web_lg --quiet

2021-06-05 19:56:11.708030: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2021-06-05 19:56:17.621155: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
2021-06-05 19:56:23.252555: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [38]:
!spacy validate

2021-06-05 19:56:29.208098: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.0.6) =================
ℹ spaCy installation: /usr/local/lib/python3.7/dist-packages/spacy

NAME              SPACY            VERSION                            
en_core_web_sm    >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_lg    >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_trf   >=3.0.0,<3.1.0   3.0.0   ✔



In [ ]:
# Train NER
!python -m spacy project run corpus spacy_target_answer

2021-04-26 19:46:40.104800: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

=================================== corpus ===================================
Running command: /usr/bin/python3 -m spacy convert /content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/quac/sentences_conll corpus -c ner
2021-04-26 19:46:45.057616: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (2302 documents): corpus/dev.spacy
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1000 documents): corpus/test.spacy
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (9263 documents): corpus/train.spacy


In [ ]:
!python -m spacy init fill-config ./spacy_target_answer/configs/base_config.cfg ./spacy_target_answer/configs/config.cfg
!python -m spacy project run train spacy_target_answer

2021-04-26 07:03:00.127155: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
spacy_target_answer/configs/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
2021-04-26 07:03:04.039780: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

=================================== train ===================================
Running command: /usr/bin/python3 -m spacy train configs/config.cfg -o training/ --gpu-id 0 --paths.train corpus/train.spacy --paths.dev corpus/dev.spacy --output /content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/training
2021-04-26 07:03:06.832328: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

In [ ]:
!python -m spacy project run evaluate spacy_target_answer

2021-04-26 13:43:10.899696: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

================================== evaluate ==================================
Running command: /usr/bin/python3 -m spacy evaluate /content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/training/model-best ./corpus/test.spacy --output /content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/metrics/config.json --gpu-id 0
2021-04-26 13:43:15.888179: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

================================== Results ==================================

TOK     -    
NER P   50.37
NER R   6.37 
NER F   11.31
SPEED   978  


=============================== NER (per type) ===============================

             P      R       F
Answer   50.37   6.37   11.31

✔ Saved results to
/content/gdrive/MyDrive/mt-qg

In [39]:
import spacy
nlp = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/training/model-best')
doc = nlp(u"There are no major roads through Golm, only state road (Landesstraße) L 902 passes through the northern parts of the village area and offers the shortest connection to federal highway (Bundesstraße) 273.  The latter can also be reached following a road extending east from Golm past Sanssouci Park to the Jägervorstadt quarter of Potsdam. The railway between Wildpark and Nauen was opened in 1902 as part of the single-tracked bypass railway (Umgehungsbahn) intended to connect the radial railway lines originating from Berlin. [...]")
spacy.displacy.render(doc, style='ent', jupyter =True)

In [40]:
nlp2 = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/squad/training/model-best')
doc = nlp2(u"There are no major roads through Golm, only state road (Landesstraße) L 902 passes through the northern parts of the village area and offers the shortest connection to federal highway (Bundesstraße) 273.  The latter can also be reached following a road extending east from Golm past Sanssouci Park to the Jägervorstadt quarter of Potsdam. The railway between Wildpark and Nauen was opened in 1902 as part of the single-tracked bypass railway (Umgehungsbahn) intended to connect the radial railway lines originating from Berlin.  [...]")
spacy.displacy.render(doc, style='ent', jupyter =True)

In [41]:
pip install scikit-learn sklearn_crfsuite -U

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)
Requirement already up-to-date: sklearn_crfsuite in /usr/local/lib/python3.7/dist-packages (0.3.6)


In [42]:
import pandas as pd
import spacy
from spacy.tokens import Doc
from tqdm import tqdm
import ast
from Custom import *
import re

spacy.require_gpu()
dataset = "squad"
df = pd.read_csv("/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/"+dataset+"/paragraph/test.csv")
df["askable_tokens"] = [ast.literal_eval(t) for t in df["askable_tokens"]]
df["paragraph_tokens"] = [ast.literal_eval(t) for t in df["paragraph_tokens"]]

print(df.columns)

nlp = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/'+dataset+'/training/model-best')
def custom_tokenizer(nlp):
    return lambda text : Doc(nlp.vocab, words=text)

nlp.tokenizer = custom_tokenizer(nlp)
nlp.add_pipe("sentencizer")


pred_y = []
true_y = []

with open("./trf_results.txt", "a") as file:
  for doc, y in tqdm(zip(nlp.pipe(df["paragraph_tokens"]), io2iob(df["askable_tokens"]))):
    labels = []
    for sent in doc.sents:
      for token, y_ in zip(sent, y[sent.start:sent.end]):
        if token.ent_iob_ =="O":
          labels.append("O")
          label_ = "O"
        else:
          labels.append(token.ent_iob_ + "-" + token.ent_type_)
          label_ = token.ent_iob_ + "-" + token.ent_type_
        file.write("{} {} {}\n".format(token.text, y_, label_))
      pred_y.append(labels)
      true_y.append(y) 
      file.write("\n")

Index(['Unnamed: 0', 'text_title', 'paragraph_text', 'paragraph_tokens',
       'askable_tokens'],
      dtype='object')



0it [00:00, ?it/s]
1it [00:01,  1.68s/it]
129it [00:04,  1.18s/it]
257it [00:07,  1.20it/s]
385it [00:10,  1.69it/s]
513it [00:12,  2.39it/s]
641it [00:15,  3.34it/s]
769it [00:17,  4.64it/s]
897it [00:19,  6.40it/s]
1025it [00:22,  8.67it/s]
1204it [00:23, 51.24it/s]


In [43]:
from sklearn_crfsuite import metrics
metrics.flat_classification_report(io2iob(true_y), pred_y, digits=5)

'              precision    recall  f1-score   support\n\n    B-Answer    0.41202   0.25524   0.31521     42999\n    I-Answer    0.47930   0.14504   0.22269    152252\n           O    0.84233   0.95940   0.89706    881750\n\n    accuracy                        0.81616   1077001\n   macro avg    0.57789   0.45322   0.47832   1077001\nweighted avg    0.77383   0.81616   0.77850   1077001\n'

In [45]:
pairs = {
  "answer_sentece_token": [],
  "answer": [],
  "answer_token": [],
  "question_token": [],
}

for doc in tqdm(nlp.pipe(df["paragraph_tokens"])):
  for sent in doc.sents:
    for entity in sent.ents:
      pairs["answer_sentece_token"].append([t.text for t in sent])
      pairs["answer"].append(entity.text)
      pairs["answer_token"].append([t.text for t in entity])
      pairs["question_token"].append([])



0it [00:00, ?it/s]

1it [00:01,  1.62s/it]

129it [00:03,  1.14s/it]

257it [00:06,  1.24it/s]

385it [00:09,  1.75it/s]

513it [00:11,  2.47it/s]

641it [00:14,  3.46it/s]

769it [00:16,  4.80it/s]

897it [00:19,  6.60it/s]

1025it [00:21,  8.94it/s]

1204it [00:22, 52.87it/s]


In [46]:
target_answer_seq = pd.DataFrame(pairs)

In [47]:
target_answer_seq.to_csv("/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/"+dataset+"/question_answer/trf_test.csv")